
# CITS4403 项目演示（AUD/USD 外汇 ABM 模型）

> **小组名称 / 组员信息 / 学号**：*（在此填写）*  
> **项目标题**：*（在此填写）*  
> **摘要（可选）**：*（2~3 句话概述你的模型与结论，稍后自行补充）*



## Notebook 说明与评分维度映射（供助教查阅）

- **可复现性**：本 Notebook 会自动解压项目代码、设置 `PYTHONPATH`，并给出一键运行的单元。  
- **方法与实现**：文字部分简述 ABM 的三类交易者与市场撮合逻辑；代码单元直接调用项目中的实现。  
- **实验与结果**：运行后生成并展示关键图表（价格、收益、滚动波动率）与指标（收益、波动率、尖峰厚尾检验等）。  
- **分析与讨论**：提供与真实数据的对比视图与要点解读模版。  
- **结构与表达**：整份文档中文讲解，版式统一，读者无需切换环境即可浏览与运行。

> *提示：小组可在“讨论/结论/局限与未来工作”处补充更深入分析与引用。*



## 0. 环境准备（自动）

- 解压上传的项目压缩包  
- 自动定位项目根目录并加入 `sys.path`  
- 列出项目结构，方便检查


In [ ]:

import os, sys, pathlib, textwrap, json, pandas as pd

PROJECT_ROOT = r"/mnt/data/workspace/project/cits4403_copy"
print("项目根目录:", PROJECT_ROOT)
assert os.path.exists(PROJECT_ROOT), "未找到项目根目录"

# 将项目根目录加入 Python 搜索路径（便于 `import src` 等）
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

# 展示关键目录结构
for p in ["src", "utils", "results", "data"]:
    pth = os.path.join(PROJECT_ROOT, p)
    print(f"存在 {p} ?", os.path.exists(pth))
    if os.path.isdir(pth):
        print(" - 子文件（最多显示前 10 个）：")
        for i, name in enumerate(sorted(os.listdir(pth))[:10], 1):
            print("   ", i, name)
        print()



## 1. 模型简介（ABM 视角）

本项目采用 **Agent‑Based Model（基于主体建模）** 来模拟 AUD/USD 外汇市场，包含三类核心交易主体：

- **投机者（Speculators）**：趋势/动量驱动，易引发短期价格偏离与波动聚集；  
- **套保者（Hedgers）**：以对冲实体风险为目的，交易方向与强度通常受敞口和风险预算驱动；  
- **基本面派（Fundamentalists）**：围绕“内在价值/基本面”进行均值回归交易，是价格回归与稳定的重要力量。

市场层面含有：
- 随机噪声 + 可能的外生冲击（News/Poisson/Shocks），模拟宏观与突发事件；
- 价格与订单的撮合/更新机制；
- 可选的 **Central Bank** 干预（若实现提供）。



## 2. 一键运行基线实验

> 说明：直接运行项目中的测试脚本（如 `test_model_improved.py`），自动生成图表与 CSV，并在此 Notebook 内部展示主要输出。


In [ ]:

import os, sys, subprocess, textwrap, pathlib

SCRIPT = None
for name in ["test_model_improved.py", "test_model.py", "run_simulation.py"]:
    p = os.path.join(PROJECT_ROOT, name)
    if os.path.exists(p):
        SCRIPT = p
        break

print("检测到可执行脚本:", SCRIPT)

# 使用当前内核直接运行脚本
if SCRIPT:
    completed = subprocess.run([sys.executable, SCRIPT], cwd=PROJECT_ROOT, capture_output=True, text=True)
    print("=== 脚本标准输出 ===\n", completed.stdout[:4000])
    if completed.stderr:
        print("\n=== 脚本标准错误（截断）===\n", completed.stderr[:2000])
else:
    print("未找到测试脚本，请将运行逻辑整合为上述三个文件之一。")



## 3. 读取并展示结果

- `results/improved_simulation.csv`：价格路径  
- `results/simulation_statistics.csv`：指标汇总  
> 若文件名不同，请在下方代码中替换。


In [ ]:

import os, pandas as pd

price_csv = os.path.join(PROJECT_ROOT, "results", "improved_simulation.csv")
stats_csv = os.path.join(PROJECT_ROOT, "results", "simulation_statistics.csv")

print("价格 CSV:", price_csv, "存在？", os.path.exists(price_csv))
print("统计 CSV:", stats_csv, "存在？", os.path.exists(stats_csv))

prices_df = pd.read_csv(price_csv) if os.path.exists(price_csv) else None
stats_df  = pd.read_csv(stats_csv) if os.path.exists(stats_csv) else None

display(prices_df.head() if prices_df is not None else "未找到价格文件")
display(stats_df if stats_df is not None else "未找到统计文件")



## 4. 可视化（价格/收益/滚动波动率）


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

if 'prices_df' in globals() and prices_df is not None:
    prices = prices_df['price'].values
    rets = np.diff(np.log(prices))
    
    fig = plt.figure(figsize=(12,8))
    ax1 = plt.subplot(3,1,1)
    ax1.plot(prices, linewidth=1.5)
    ax1.set_title("模拟价格")
    ax1.set_ylabel("Price")
    ax1.grid(True, alpha=0.3)
    
    ax2 = plt.subplot(3,1,2)
    ax2.plot(rets, linewidth=1.0)
    ax2.axhline(0, ls="--")
    ax2.set_title("对数收益")
    ax2.grid(True, alpha=0.3)
    
    ax3 = plt.subplot(3,1,3)
    rolling_vol = pd.Series(rets).rolling(20).std()
    ax3.plot(rolling_vol, linewidth=1.5)
    ax3.fill_between(range(len(rolling_vol)), rolling_vol, alpha=0.3)
    ax3.set_title("滚动波动率（20）")
    ax3.set_xlabel("Time")
    ax3.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("未能读取到价格数据，无法绘图。")



## 5. 与真实数据对比（可选）

若 `data/merged_data.csv` 存在且包含 `AUD_USD_Close` 列，将对比真实与模拟的价格与收益分布。


In [ ]:

import os, pandas as pd, numpy as np, matplotlib.pyplot as plt

data_path = os.path.join(PROJECT_ROOT, "data", "merged_data.csv")
if os.path.exists(data_path) and 'prices' in globals():
    real = pd.read_csv(data_path, index_col=0, parse_dates=True)
    if 'AUD_USD_Close' in real.columns:
        sim_steps = min(len(prices), len(real))
        real_prices = real['AUD_USD_Close'].values[:sim_steps]
        real_rets = np.diff(np.log(real_prices))
        sim_rets  = np.diff(np.log(prices[:sim_steps]))
        
        fig, axes = plt.subplots(2,2, figsize=(12,8))
        axes[0,0].plot(real_prices); axes[0,0].set_title("真实价格")
        axes[0,1].plot(prices[:sim_steps]); axes[0,1].set_title("模拟价格")
        axes[1,0].hist(real_rets, bins=50, alpha=0.7); axes[1,0].set_title("真实收益分布")
        axes[1,1].hist(sim_rets, bins=50, alpha=0.7); axes[1,1].set_title("模拟收益分布")
        for ax in axes.flatten(): ax.grid(True, alpha=0.3)
        plt.tight_layout(); plt.show()
    else:
        print("未找到列 AUD_USD_Close，跳过对比。")
else:
    print("未找到数据文件或价格数组，跳过对比。")



## 6. 结果分析（撰写模版）

- **总体走势**：*（描述价格趋势、波动聚集现象）*  
- **统计特性**：*（均值/方差/偏度/峰度；与正态分布的差异；是否存在厚尾）*  
- **机制解释**：*（从投机者/套保者/基本面派与外生冲击的角度解释波动来源）*  
- **与真实数据的差异**：*（在哪些方面更接近/仍待改进，可能原因）*  
- **消融/敏感性实验（可选）**：*（例如改变基本面人数、冲击强度、到达率等，观察指标变化）*  



## 7. 局限与未来工作（撰写模版）

- **局限**：*（例如参数较多、标定依赖、市场微观结构简化等）*  
- **未来工作**：*（引入更真实的订单簿/做市商，GARCH/ARIMA 混合波动，利用宏观/商品价格等多源数据进行校准）*



## 8. 复现实验指南（面向助教）

1. 运行本 Notebook 自带的“0. 环境准备”单元；  
2. 直接运行“2. 一键运行基线实验”；  
3. 在“3. 读取并展示结果”“4/5. 可视化/对比”中查看输出图表与指标；  
4. 若需更换参数与情景，可到项目根目录下编辑 `test_model_improved.py` 或相应脚本后重新运行。

> 注：若在你机房的 Python 版本缺组件，可根据项目的 `requirements.txt` 安装。



## 附：项目结构（自动生成）

- `src/`：ABM 模型主体与交易者类型  
- `utils/`：可视化/工具函数  
- `results/`：运行产生的 CSV 与图像  
- `data/`：原始/清洗数据（若提供）

> *请保持该结构，以便 Notebook 自动发现与运行。*
